This notebook will show how to calibrate a Normal Hydrogen reference Electrode (NHE) in alkaline environment (pH larger than 8).

In [ ]:
import pandas as pd


Now it is time to load the data. This repository contains a .csv with data from a Cyclic Voltammetry scan of platinum wire vs. platinum wire with a NHE as reference electrode, performed in 1M KOH at room temperature. Ph = 14.

The parameters of the CV scan was:
    init_voltage=0.45, # V
    final_voltage=0.45, # V
    apex1=1.4, # V
    apex2=0, # V
    stepsize=0.001, # Resolution in Volts
    scanrate=1, # V/s
    cycles=20, # We will use scan cycle 19

In [ ]:
# Load the potentiostat data from .csv file with headers
csv_file_path = 'data.csv'
df_pt = pd.read_csv(csv_file_path, header=0)

In [ ]:
# Define the name of the columns containing the current (I) and potential (V) data
current_column = 'Current (A)'
potential_column = 'Potential (V)'

In [ ]:
def get_platinum_potential(df_pt: pd.DataFrame, ohmic_resistance: float):
    """Measure the potential of the platinum electrode

    Args:
        ohmic_resistance (float): Ohmic resistance of the platinum electrode

    Returns:
        df_pt_smoothed (pd.DataFrame): Smoothed data of the platinum electrode
        pt_peak_potential_ohmic_corrected (float): Peak potential of the platinum electrode
        df_pt (pd.DataFrame): Original data of the platinum electrode
    """

    # Set column headers
    df_pt = set_column_headers_cv(df_pt)
    logging.debug(f"df_pt: {df_pt.to_string}")

    # Correct for ohmic resistance
    df_pt = correct_for_ohmic_resistance(df_pt, ohmic_resistance, OHMIC_CORRECTION_FACTOR)

    # Select part of the data
    df_pt_filtered = filter_data(
        df_pt,
        lower_potential=0.65,
        upper_potential=1.2,  # It is cut further in the smoothing function
        first_cycle=19,
        last_cycle=19,
    )
    logging.debug(f"df_pt: {df_pt_filtered.to_string}")

    # Smooth data
    # df_pt_smoothed, pt_peak_potential_ohmic_corrected = smooth_data_lowess(df_pt_filtered)
    df_pt_smoothed, pt_peak_potential_ohmic_corrected = smooth_data_savitzky_golay(df_pt_filtered)
    logging.debug(f"pt_peak_potential_ohmic_corrected: {pt_peak_potential_ohmic_corrected}")
    logging.debug(f"df_pt_smoothed: {df_pt_smoothed.to_string}")

    # Give original a name in the column "Category"
    df_pt["Category"] = "Measurement"

    # Merge data
    # logging.debug("Merging smoothed CV data and original ")
    # df_merged = pd.concat([df_pt, df_pt_smoothed], ignore_index=True)

    return df_pt_smoothed, pt_peak_potential_ohmic_corrected, df_pt